In [1]:
import pandas as pd

In [2]:
# Dataset da variavel target
df_aprovados = pd.read_csv('https://raw.githubusercontent.com/FIAP/Pos_Tech_DTAT/dd201a034223a16732c6f639b40600b26bd3129c/dados/clientes_aprovados.csv')

# Dataset tratado no primeiro script
df_cadastrados = pd.read_csv('../../dados/credit_score/clientes_cadastrados_clean.csv')

In [4]:
# ID Cliente -> Join com df_cadastrados
# Mes_referencia -> 0 = mês atual, -1 = mês passado, etc.
# Faixa_atraso -> quanto tempo atrasou o pagamento
df_aprovados.head()

,ID_Cliente,Mes_referencia,Faixa_atraso
0,5001711,0,nenhum empréstimo
1,5001711,-1,1-29 dias
2,5001711,-2,1-29 dias
3,5001711,-3,1-29 dias
4,5001712,0,pagamento realizado


In [5]:
# Valores possíveis para Faixa_atraso
df_aprovados['Faixa_atraso'].value_counts()

Faixa_atraso
pagamento realizado    442031
1-29 dias              383120
nenhum empréstimo      209230
30-59 dias              11090
>150 dias                1693
60-89 dias                868
90-119 dias               320
120-149 dias              223
Name: count, dtype: int64

In [7]:
# Exemplo de linhas para um cliente específico
df_aprovados.loc[df_aprovados['ID_Cliente'] == 5001712]

,ID_Cliente,Mes_referencia,Faixa_atraso
4,5001712,0,pagamento realizado
5,5001712,-1,pagamento realizado
6,5001712,-2,pagamento realizado
7,5001712,-3,pagamento realizado
8,5001712,-4,pagamento realizado
9,5001712,-5,pagamento realizado
10,5001712,-6,pagamento realizado
11,5001712,-7,pagamento realizado
12,5001712,-8,pagamento realizado
13,5001712,-9,1-29 dias


**Variável target**: cliente é bom ou mau pagador?

Isso requere etapa de feature engineering, ponderar diversos fatores!

# Feature Engineering

## Mês de abertura e fechamento das contas

In [13]:
df_goupby_id = df_aprovados.groupby('ID_Cliente').agg(Mes_inicio_conta=('Mes_referencia','min'), Mes_fim_conta = ('Mes_referencia','max')).reset_index()
df_goupby_id

,ID_Cliente,Mes_inicio_conta,Mes_fim_conta
0,5001711,-3,0
1,5001712,-18,0
2,5001713,-21,0
3,5001714,-14,0
4,5001715,-59,0
...,...,...,...
45980,5150482,-28,-11
45981,5150483,-17,0
45982,5150484,-12,0
45983,5150485,-1,0


In [14]:
# Merge no df aprovados
df_aprovados_mes = df_aprovados.merge(df_goupby_id, on='ID_Cliente')
df_aprovados_mes

,ID_Cliente,Mes_referencia,Faixa_atraso,Mes_inicio_conta,Mes_fim_conta
0,5001711,0,nenhum empréstimo,-3,0
1,5001711,-1,1-29 dias,-3,0
2,5001711,-2,1-29 dias,-3,0
3,5001711,-3,1-29 dias,-3,0
4,5001712,0,pagamento realizado,-18,0
...,...,...,...,...,...
1048570,5150487,-25,pagamento realizado,-29,0
1048571,5150487,-26,pagamento realizado,-29,0
1048572,5150487,-27,pagamento realizado,-29,0
1048573,5150487,-28,pagamento realizado,-29,0


## Número de meses

In [15]:
df_aprovados_mes['N_meses'] = df_aprovados_mes['Mes_fim_conta'] - df_aprovados_mes['Mes_inicio_conta']

In [16]:
df_aprovados_mes

,ID_Cliente,Mes_referencia,Faixa_atraso,Mes_inicio_conta,Mes_fim_conta,N_meses
0,5001711,0,nenhum empréstimo,-3,0,3
1,5001711,-1,1-29 dias,-3,0,3
2,5001711,-2,1-29 dias,-3,0,3
3,5001711,-3,1-29 dias,-3,0,3
4,5001712,0,pagamento realizado,-18,0,18
...,...,...,...,...,...,...
1048570,5150487,-25,pagamento realizado,-29,0,29
1048571,5150487,-26,pagamento realizado,-29,0,29
1048572,5150487,-27,pagamento realizado,-29,0,29
1048573,5150487,-28,pagamento realizado,-29,0,29


## MOB

Months on Book -> para cada mês de referência, indica há quantos meses o relacionamento com o cliente se iniciou

MOB = Mes_referencia - Mes_inicio_conta

In [17]:
df_aprovados_mob = df_aprovados_mes.copy()

In [18]:
df_aprovados_mob['MOB'] = df_aprovados_mob['Mes_referencia'] - df_aprovados_mob['Mes_inicio_conta']

In [19]:
df_aprovados_mob

,ID_Cliente,Mes_referencia,Faixa_atraso,Mes_inicio_conta,Mes_fim_conta,N_meses,MOB
0,5001711,0,nenhum empréstimo,-3,0,3,3
1,5001711,-1,1-29 dias,-3,0,3,2
2,5001711,-2,1-29 dias,-3,0,3,1
3,5001711,-3,1-29 dias,-3,0,3,0
4,5001712,0,pagamento realizado,-18,0,18,18
...,...,...,...,...,...,...,...
1048570,5150487,-25,pagamento realizado,-29,0,29,4
1048571,5150487,-26,pagamento realizado,-29,0,29,3
1048572,5150487,-27,pagamento realizado,-29,0,29,2
1048573,5150487,-28,pagamento realizado,-29,0,29,1


## Cliente ativo

In [20]:
df_aprovados_mob['Cliente_ativo'] = df_aprovados_mob['Mes_fim_conta'].apply(lambda x: 1 if x == 0 else 0)

In [21]:
df_aprovados_mob

,ID_Cliente,Mes_referencia,Faixa_atraso,Mes_inicio_conta,Mes_fim_conta,N_meses,MOB,Cliente_ativo
0,5001711,0,nenhum empréstimo,-3,0,3,3,1
1,5001711,-1,1-29 dias,-3,0,3,2,1
2,5001711,-2,1-29 dias,-3,0,3,1,1
3,5001711,-3,1-29 dias,-3,0,3,0,1
4,5001712,0,pagamento realizado,-18,0,18,18,1
...,...,...,...,...,...,...,...,...
1048570,5150487,-25,pagamento realizado,-29,0,29,4,1
1048571,5150487,-26,pagamento realizado,-29,0,29,3,1
1048572,5150487,-27,pagamento realizado,-29,0,29,2,1
1048573,5150487,-28,pagamento realizado,-29,0,29,1,1


## Faixa_atraso

Mudar Faixa_atraso para valores numéricos

In [22]:
df_aprovados_faixa_atraso = df_aprovados_mob.copy()

In [24]:
df_aprovados_faixa_atraso['Faixa_atraso'].unique()

array(['nenhum empréstimo', '1-29 dias', 'pagamento realizado',
       '30-59 dias', '60-89 dias', '90-119 dias', '120-149 dias',
       '>150 dias'], dtype=object)

In [25]:
dict_faixa_atraso = {'nenhum empréstimo':0, 'pagamento realizado':1,'1-29 dias':2,
       '30-59 dias':3, '60-89 dias':4, '90-119 dias':5, '120-149 dias':6,
       '>150 dias':7}

In [28]:
df_aprovados_faixa_atraso['Int_Faixa_atraso'] = df_aprovados_faixa_atraso['Faixa_atraso'].map(dict_faixa_atraso)
df_aprovados_faixa_atraso.drop(columns=['Faixa_atraso'], inplace=True)
df_aprovados_faixa_atraso.rename(columns={'Int_Faixa_atraso':'Faixa_atraso'}, inplace=True)

## Variável Bom/Mau pagador

In [35]:
df_aprovados_pagadores = df_aprovados_faixa_atraso.copy()

In [36]:
df_aprovados_pagadores['Mau_pagador'] = df_aprovados_pagadores['Faixa_atraso'].apply(lambda x: 1 if x > 2 else 0)

In [37]:
df_aprovados_pagadores

,ID_Cliente,Mes_referencia,Mes_inicio_conta,Mes_fim_conta,N_meses,MOB,Cliente_ativo,Faixa_atraso,Mau_pagador
0,5001711,0,-3,0,3,3,1,0,0
1,5001711,-1,-3,0,3,2,1,2,0
2,5001711,-2,-3,0,3,1,1,2,0
3,5001711,-3,-3,0,3,0,1,2,0
4,5001712,0,-18,0,18,18,1,1,0
...,...,...,...,...,...,...,...,...,...
1048570,5150487,-25,-29,0,29,4,1,1,0
1048571,5150487,-26,-29,0,29,3,1,1,0
1048572,5150487,-27,-29,0,29,2,1,1,0
1048573,5150487,-28,-29,0,29,1,1,1,0


## Número de vezes que foi mau pagador

In [42]:
df_groupby_n_mau_pagador = df_aprovados_pagadores.groupby('ID_Cliente').agg(N_Mau_pagador = ('Mau_pagador','sum')).reset_index()
df_groupby_n_mau_pagador

,ID_Cliente,N_Mau_pagador
0,5001711,0
1,5001712,0
2,5001713,0
3,5001714,0
4,5001715,0
...,...,...
45980,5150482,0
45981,5150483,0
45982,5150484,0
45983,5150485,0


In [43]:
df_aprovados_pagadores = df_aprovados_pagadores.merge(df_groupby_n_mau_pagador, on='ID_Cliente')

In [44]:
df_aprovados_pagadores

,ID_Cliente,Mes_referencia,Mes_inicio_conta,Mes_fim_conta,N_meses,MOB,Cliente_ativo,Faixa_atraso,Mau_pagador,N_Mau_pagador
0,5001711,0,-3,0,3,3,1,0,0,0
1,5001711,-1,-3,0,3,2,1,2,0,0
2,5001711,-2,-3,0,3,1,1,2,0,0
3,5001711,-3,-3,0,3,0,1,2,0,0
4,5001712,0,-18,0,18,18,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...
1048570,5150487,-25,-29,0,29,4,1,1,0,0
1048571,5150487,-26,-29,0,29,3,1,1,0,0
1048572,5150487,-27,-29,0,29,2,1,1,0,0
1048573,5150487,-28,-29,0,29,1,1,1,0,0


## Proporção de meses como mau pagador

In [51]:
df_aprovados_pagadores['Ratio_Mau_pagador'] = df_aprovados_pagadores.apply(lambda x: (x['N_Mau_pagador'] / x['N_meses']) * 100 if x['N_meses'] > 0 else 0, axis=1)

In [54]:
df_aprovados_pagadores['Ratio_Mau_pagador'].value_counts()

Ratio_Mau_pagador
0.000000     904765
5.263158       2682
5.000000       2647
7.142857       2596
5.882353       2530
              ...  
92.857143        15
53.846154        14
84.615385        14
72.727273        12
87.500000         9
Name: count, Length: 395, dtype: int64

# Criação da variável target

In [61]:
df_aprovados_pagadores.loc[df_aprovados_pagadores['ID_Cliente'] == 5001717]

,ID_Cliente,Mes_referencia,Mes_inicio_conta,Mes_fim_conta,N_meses,MOB,Cliente_ativo,Faixa_atraso,Mau_pagador,N_Mau_pagador,Ratio_Mau_pagador
120,5001717,0,-21,0,21,21,1,1,0,0,0.0
121,5001717,-1,-21,0,21,20,1,1,0,0,0.0
122,5001717,-2,-21,0,21,19,1,1,0,0,0.0
123,5001717,-3,-21,0,21,18,1,1,0,0,0.0
124,5001717,-4,-21,0,21,17,1,1,0,0,0.0
125,5001717,-5,-21,0,21,16,1,2,0,0,0.0
126,5001717,-6,-21,0,21,15,1,2,0,0,0.0
127,5001717,-7,-21,0,21,14,1,2,0,0,0.0
128,5001717,-8,-21,0,21,13,1,2,0,0,0.0
129,5001717,-9,-21,0,21,12,1,2,0,0,0.0
